<a href="https://colab.research.google.com/github/veni-vidi-vici-tech/Fragment_Intensity_Prediction/blob/main/Fragment_Intensity_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fragment Intensity Prediction

**GOAL**: Develop a machine learning model to predict Fragment ion intensities of y and b singly charged ions.

Example:
* Input: </br>
Peptide Sequence: LTQETNRV, Precursor Charge: 4 
* Output: 

  <img src='pictures_and_graphs/desired_output.png' width=600>




## I. Data Analysis

* 74,817,546 initial entries

* 7 columns with no missing values:

  <img src='pictures_and_graphs/data_analysis.png' width=200>


###peptide_sequence column

* 393.369 unique sequences
* 7-30 amino acid length range
<img src='pictures_and_graphs/average_peptide_length.png' width=500>

* max number of occurances: 31.516 (HWYITTGPVREK)
* min number of occurances: 7 (QQQQQQQQQQQQRR)
* average number of occurances per sequence: 190

* There are two types of UNIMOD modifications: C[UNIMOD:4] and M[UNIMOD:35]
* Cytosine is *always* UNIMOD:4 modified, while Methionine can be UNIMOD:35 modified or not

###raw file and scan_number columns
* One spectrum: raw_file and scan_number stay the same
* **UNIQUE IDENTIFIER: peptide_sequence + raw_file + scan_number**

###precursor charge

  <img src='pictures_and_graphs/precursor_charge.png' width=200>

* The most common precursor_charge values are 2 and 3
* Some sequences have several different precursor_charge values

###ion type and no columns
* There are two types of ions: b and y.
* There are more y ions (37,404,506) than b ions (36,745,591)
* 1-28 ion no range

###intensity
<img src='pictures_and_graphs/avarage_intensity.png' width=600>

* values need to be normalized




## II. Data Preprocessing

###1. Removing Confusing Values

- Data was merged from several files
- There are 74,817,546 entries in our dataset at the beginning
- **GOAL**: remove errors and noise from the data

In [ ]:
def clean_data(df):
  ''' 
  Checks the DataFrame for errors and noise and 
  returns it without them
  '''
  # 2.1 Removing join errors
  df['precursor_max'] = df.groupby(['peptide_sequence', 'scan_number', 'raw_file'])['precursor_charge'].transform(max)
  df['precursor_min'] = df.groupby(['peptide_sequence', 'scan_number', 'raw_file'])['precursor_charge'].transform(min)
  df.loc[df['precursor_max'] != df['precursor_min']]
  df.drop(df.index[df['precursor_max'] != df['precursor_min']], inplace = True)
  df.drop(['precursor_max', 'precursor_min'], axis=1, inplace=True)

  # 2.2 Removing noise from the data
  df.drop(df.index[df["peptide_sequence"].str.len() == df['no']], inplace = True)
  return df

* After applying clean_data function 667958 rows were removed from the dataset

###2. Peptide_Sequence Column Cleaning

* `peptide_sequence` column contains UNIMOD modifications:
  *  `C[UNIMOD:4]` = cystein is **always modified** (in carboxyamidomethylated form)
  *  `M[UNIMOD:35]` = methionine is **in some cases modified** (oxidized to methionine sulfoxide) 

* **GOAL**: make the data in 'peptide_sequence' column consistent by removing [UNIMOD:4] and replacing [UNIMOD:35] with 'O'

In [ ]:
df['peptide_sequence'] = df['peptide_sequence'].apply(lambda x: x.replace('[UNIMOD:4','')
                                                                 .replace(']','')
                                                                 .replace('M[UNIMOD:35', 'O'))

#### Exploring intensities of neighbouring amino acids

* The `b and y ions` for a given peptide represent the two halves formed by splitting the original peptide at a **peptide bond** between two amino acids.

<div>
<img src="pictures_and_graphs/b_and_y_ions.JPG" width="600"/>
</div>

<img src='pictures_and_graphs/neighbouring_aas.png'>

* **Proline** at *C-Terminus* tends to have high intensity with almost all neighboring amino acids
* **Arginine** tends to have have intensity with neighboring Arginine

###3. Normalization

- Big fluctuations between `intensity` values 
- **GOAL**: normalize `intensity` column values (values between 0.0 and 1.0)

In [ ]:
def normalize_intensity(df):
  '''
  Takes intensity values from the DataFrame, normalizes them using 
  cumulative intensity normalization method and returns the DataFrame 
  with the new normalized values in a ['normalized_intensity'] column
  '''
  # 1. Assigning rank to every intensity in the spectra
  df['rank'] = df.groupby(['peptide_sequence', 'scan_number', 'raw_file'])["intensity"].rank("dense", ascending=False)

  # 2. Calculating intensity total within each spectra
  df['total'] = df.groupby(['peptide_sequence', 'scan_number', 'raw_file'])["intensity"].transform('sum')

  # 3. Sorting values within the spectra based on their rank
  df['sum_greater_than'] = df.sort_values('rank', ascending=False).groupby(['peptide_sequence', 'scan_number', 'raw_file'])['intensity'].cumsum()

  # 4. Calculating the normalized intensity values within the spectra
  df['normalized_intensity'] = df['sum_greater_than']/df['total']

  # 5. Dropping helper-columns
  df.drop(['rank', 'sum_greater_than', 'total'], axis = 1, inplace = True) 
  df
  return df

<img src='pictures_and_graphs/norm_intensity.png' width=600>

###4. Turning data into numbers

* `ion_type`, `no` and `intensity` are target features and need to be part of the same NumPy array structured like so:

  * index 0-27 => ion type b 1-28
  * index 28-55 => ion type y 1-28

* `peptide_sequence` column needs to be converted into numbers using dictionary:

  <img src="pictures_and_graphs/amino_acid_key.png" 
  width="100" /> 


In [ ]:
import numpy as np
def create_target():
  df['group'] = df.groupby(['peptide_sequence', 'scan_number', 'raw_file']).ngroup()
  # df['group'] = df.groupby(['peptide_sequence', 'precursor_charge']).ngroup() 
  groups = df.groupby('group')
  result = []
  for name, group in groups:
    intensities = np.zeros(56)
    ion_groups = group.groupby('ion_type')

    for ion_name, ion_group in ion_groups:
      if ion_name == 'b':
        indices = ion_group['no'] - 1
        #intensities[indices] = ion_group['mean_normalized_intensity'].values
        intensities[indices] = ion_group['normalized_intensity'].values
      else:
        indices = 27 + ion_group['no']
        #intensities[indices] = ion_group['mean_normalized_intensity'].values
        intensities[indices] = ion_group['normalized_intensity'].values
    result.append(intensities)

In [ ]:
# Create 'peptide_length' column for further use
df['peptide_length'] = df['peptide_sequence'].apply(len)

In [ ]:
def encode_peptides(df):

  # 1. Make sure that all the sequences have the same length
  # Finding the longest sequence
  max_len = df['peptide_sequence'].str.len().max()
  # Addding dummy values to shorter sequences to achieve the same length for every sequence
  df['peptide_sequence_encoded'] = df['peptide_sequence'].apply(lambda x: x + 'X'*(max_len - len(x)))
  # 2. Separate all the letters with comma
  df['peptide_sequence_encoded'] = df['peptide_sequence_encoded'].agg(lambda x: ','.join(x))
  # 3. Replace one-letter code with numbers
  # Creating a dictionary, where dummy value X, every amino acid and 'O'(oxidized methionine) is numbered
  aa_dict = {'X' : '0',
             'A' : '1', 'C' : '2', 'D' : '3',
             'E' : '4', 'F' : '5', 'G' : '6', 
             'H' : '7', 'I' : '8', 'K' : '9', 
             'L' : '10', 'M' : '11', 'N' : '12', 
             'P' : '13', 'Q' : '14', 'R' : '15',
             'S' : '16', 'T' : '17', 'V' : '18',
             'W' : '19', 'Y' : '20', 'O' : '21'}
  # Iterating over all key-value pairs in aa_dict dictionary
  for key, value in aa_dict.items():
      # Replace key character(one letter code) with value character(number) in a sequence
      df['peptide_sequence_encoded'] = df['peptide_sequence_encoded'].apply(lambda x: x.replace(key, value))

  # 4. Turn string of numbers into integers
  df['peptide_sequence_encoded'] = df['peptide_sequence_encoded'].apply(lambda x: [int(i) for i in x.split(",")])

###5. Splitting the data

* Some `peptide_sequence` values have several `precursor charge` values and need to be in the same split
* **GOAL**: 
  * data split into 20% test_set + 80% training
  * training further split into 5 cross-validation splits: 80% train + 20% validation
  

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
np.random.seed(19)
def split_train_test(df, test_size):
  '''
  Splits the df into train and test splits
  '''

  # Create a new column in your dataframe that groups rows with the same peptide sequence value
  df['peptide_group'] = df.groupby('peptide_sequence').ngroup()

  # Create an instance of GroupShuffleSplit
  gss = GroupShuffleSplit(n_splits=1, test_size=test_size)

  # Split your dataframe into train and test sets using the iterator
  for train_index, test_index in gss.split(df, groups=df['peptide_group']):
      training = df.iloc[train_index]
      test_df = df.iloc[test_index]

  return training, test_df

In [ ]:
def find_common_peptides(df_1, df_2):
  '''
  Finds common peptides between df_1 and df_2
  '''
  df_1_peptides = set(df_1['peptide_sequence'])
  df_2_peptides = set(df_2['peptide_sequence'])

  common_peptides = df_1_peptides & df_2_peptides
  print(common_peptides)

In [ ]:
def calculate_precursor_percentage(splitted_df, original_df):
  '''
  Calculates the percentage of precursor charge values in the train split
  '''
  precursor_charge_counts = splitted_df['precursor_charge'].value_counts()
  return ((precursor_charge_counts / original_df['precursor_charge'].value_counts()) * 100)


def calculate_peptide_length_percentage(splitted_df, original_df):
  '''
  Calculates the percentage of precursor charge values in the train split
  '''
  peptide_length_counts = splitted_df['peptide_length'].value_counts()
  return ((peptide_length_counts / original_df['peptide_length'].value_counts()) * 100)

In [ ]:
def save_test(test_df):
  '''
  Transforms test_df into right format 
  and saves it as 3 NumPy arrays
  '''
  # Transforming data into numpy array
  test_pre = test_df[["precursor_charge"]].to_numpy()
  test_int = test_df[["target"]].to_numpy()
  test_seq = test_df[["peptide_sequence_encoded"]].to_numpy()

  test_seq = np.array(test_seq).flatten()
  test_int = np.array(test_int).flatten()
  test_pre = test_pre.flatten()

  test_seq = np.stack(test_seq, axis=0)
  test_int = np.stack(test_int, axis=0)

  np.save("test_pre.npy", test_pre)
  np.save("test_int.npy", test_int)
  np.save("test_seq.npy", test_seq)

In [ ]:
# Create cross-validation splits
from sklearn.model_selection import GroupKFold
from sklearn.utils import shuffle
np.random.seed(19)

def split_cross_val(training):
  '''
  Splits training df into 5 cross-validation splits and saves them
  '''

  splits=[]

  # shuffle the rows of the training dataframe
  training = shuffle(training)

  gkf = GroupKFold(n_splits=5)
  for train_index, valid_index in gkf.split(training, groups=training['peptide_group']):
      train_df = training.iloc[train_index]
      valid_df = training.iloc[valid_index]
      splits.append((train_df, valid_df))

  s_train_list = []
  s_valid_list = []

  for i in range(5):
      s_train = splits[i][0]
      s_valid = splits[i][1]
      s_train_list.append(s_train)
      s_valid_list.append(s_valid)


  for i in range(5):
    s_train = s_train_list[i]
    s_valid = s_valid_list[i]
    
    s_train_pre = s_train[["precursor_charge"]].to_numpy()
    s_train_int = s_train[["target"]].to_numpy()
    s_train_seq = s_train[["peptide_sequence_encoded"]].to_numpy()
    s_train_seq = np.array(s_train_seq).flatten()
    s_train_int = np.array(s_train_int).flatten()
    s_train_seq = np.stack(s_train_seq, axis=0)
    s_train_int = np.stack(s_train_int, axis=0)
    s_train_pre = s_train_pre.flatten()

    s_valid_pre = s_valid[["precursor_charge"]].to_numpy()
    s_valid_int = s_valid[["target"]].to_numpy()
    s_valid_seq = s_valid[["peptide_sequence_encoded"]].to_numpy()
    s_valid_seq = np.array(s_valid_seq).flatten()
    s_valid_int = np.array(s_valid_int).flatten()
    s_valid_seq = np.stack(s_valid_seq, axis=0)
    s_valid_int = np.stack(s_valid_int, axis=0)
    s_valid_pre = s_valid_pre.flatten()

    np.save(f"s{i}_train_pre.npy", s_train_pre)
    np.save(f"s{i}_train_int.npy", s_train_int)
    np.save(f"s{i}_train_seq.npy", s_train_seq)
    np.save(f"s{i}_valid_pre.npy", s_valid_pre)
    np.save(f"s{i}_valid_int.npy", s_valid_int)
    np.save(f"s{i}_valid_seq.npy", s_valid_seq)

## III. Building a Model
* **GOAL:** Create a model, which 
  * takes two inputs
  * has max 2 convolutional layers
  * maximizes spectral angle value

In [6]:
def spectral_angle_loss(y_true, y_pred):
  import keras.backend as k
  import tensorflow
  # Normalize the vectors
  x = k.l2_normalize(y_true, axis=-1)
  y = k.l2_normalize(y_pred, axis=-1)

  # Calculate the dot product between the vectors
  dot_product = k.sum(x * y, axis=-1)

  # Return the spectral angle
  return -(1 - 2 * tensorflow.acos(dot_product) / np.pi )

### Model 1
* takes two one-hot encoded inputs:
    * input A: precursor charge
    * input B: peptide sequence
* outputs 56-dimensional tensor where first 28 elements are b ions, last 28 elements y ions

In [ ]:
# Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(50, 7, activation='relu')(seq_input)

  # Concatenate layers
  concat = layers.concatenate([dense, x], axis=-1)
  x = layers.Dense(112, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='model1')

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 7), (None, 24, 64)]

### Model 2

- added Flatten layer to fix the error

In [ ]:
  # Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(50, 7, activation='relu')(seq_input)
  flatten = layers.Flatten()(x)

  # Concatenate layers
  concat = layers.concatenate([dense, flatten], axis=-1)

  x = layers.Dense(112, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='model2')

### Model 3

- added another Convolutional layer

In [ ]:
  # Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(50, 7, activation='relu')(seq_input)
  x = layers.Conv1D(300, 7, activation='relu')(x)
  flatten = layers.Flatten()(x)

  # Concatenate layers
  concat = layers.concatenate([dense, flatten], axis=-1)

  x = layers.Dense(112, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='model3')

### Model 4

* added MaxPooling1D layer

In [ ]:
  # Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(50, 7, activation='relu')(seq_input)
  x = layers.Conv1D(300, 7, activation='relu')(x)
  x = layers.MaxPooling1D()(x)
  flatten = layers.Flatten()(x)

  # Concatenate layers
  concat = layers.concatenate([dense, flatten], axis=-1)

  x = layers.Dense(112, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='model4')

### Model 5

* added BatchNormalization layers

In [ ]:
  # Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(64, 7, activation='relu')(seq_input)
  x = layers.BatchNormalization()(x)
  x = layers.Conv1D(256, 5, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling1D()(x)
  flatten = layers.Flatten()(x)

  # Concatenate layers
  concat = layers.concatenate([dense, flatten], axis=-1)

  x = layers.Dense(112, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='model_5')

### Hyperparameter tuning using Weights&Biases Framework

https://api.wandb.ai/links/annah07/yhl8pw2v

### Final Model Architecture
<img src = 'pictures_and_graphs/my_model.jpg' width=700>

In [7]:
import tensorflow as tf
import numpy as np
from keras import layers
from keras import Input
from keras.models import Model
from keras.utils import to_categorical
from keras import optimizers 

def create_model():

  # Adding Input A (precursor charge)
  precursor_input = Input(shape=(7,), name='precursor')
  dense = layers.Dense(7, activation='relu')(precursor_input)

  # Adding Input B (peptide sequence)
  seq_input = Input(shape=(30,22), name='sequence')
  x = layers.Conv1D(64, 7, activation='relu')(seq_input)
  x = layers.BatchNormalization()(x)
  x = layers.Conv1D(256, 5, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling1D()(x)
  flatten = layers.Flatten()(x)

  # Concatenate layers
  concat = layers.concatenate([dense, flatten], axis=-1)

  x = layers.Dense(128, activation='relu')(concat)
  output = layers.Dense(56, activation='sigmoid')(x)

  model = Model([precursor_input, seq_input], output, name='final_model')

  return model

def compile_model(model):
  model.compile(
      optimizer = optimizers.Adam(),
      loss = spectral_angle_loss,
      metrics = ['cosine_similarity', spectral_angle_loss])

def load_and_fit (model, num_epochs=5, batch_size=128):
  for i in range(0, 5):
      train_int = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_train_int.npy") 
      train_pre = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_train_pre.npy")
      train_seq = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_train_seq.npy")
      valid_int = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_valid_int.npy")
      valid_pre = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_valid_pre.npy")
      valid_seq = np.load(f"/content/drive/MyDrive/daniel_splits/split{i}/s{i}_valid_seq.npy")

      # One-hot encode the peptide sequences and the intensities
      train_seq = tf.one_hot(train_seq, 22)
      train_pre = tf.one_hot(train_pre, 7)
      valid_seq = tf.one_hot(valid_seq, 22)
      valid_pre = tf.one_hot(valid_pre, 7)

      # Train the model on the current cross-validation set
      model.fit(x=[train_pre, train_seq], 
                y=train_int,
                epochs=num_epochs, 
                batch_size=batch_size,
                validation_data=([valid_pre, valid_seq], [valid_int]))

      # Free the memory by deleting the one-hot encoded data
      del train_seq, train_pre, train_int, valid_seq, valid_pre, valid_int
      tf.keras.backend.clear_session()
  return model

In [ ]:
import tensorflow as tf
import numpy as np

def load_and_predict(model):
  # Load Holdout Set
  # Input 
  hold_pre = np.load('/content/drive/MyDrive/splits/holdout/test_pre.npy')
  hold_seq = np.load('/content/drive/MyDrive/splits/holdout/test_seq.npy')
  hold_int = np.load('/content/drive/MyDrive/splits/holdout/test_int.npy')

  # One-Hot-Encoding
  # Sequence
  X_hold_seq = tf.one_hot(hold_seq, depth=22)
  # Precursor Charge
  X_hold_pre = tf.one_hot(hold_pre, depth=7)
  predictions = model.predict([X_hold_pre, X_hold_seq])
  return predictions

**RESULTS**

<img src='pictures_and_graphs/performance.png' width=400>

In [6]:
from scipy import spatial
def spectral_angle(x, y):
  '''
  Calculates and returns a single spectral angle value
  '''
  x_norm = np.linalg.norm(x)
  y_norm = np.linalg.norm(y)
  prod = np.dot(x/x_norm, y/y_norm)
  if prod > 1.0:
    prod = 1
  return 1-2*(np.arccos(prod)/np.pi)

def calculate_spectral_angle(reference, holdout):
  '''
  Returns a numpy array of spectral angle values for reference and holdout
  '''
  spectral_angle_vals = []
  for i in range(holdout.shape[0]):
    holdout_value = holdout[i,:]
    reference_value = reference[i,:]
    spectral_angle_value = spectral_angle(holdout_value, reference_value)
    spectral_angle_vals.append(spectral_angle_value)
  return np.array(spectral_angle_vals)

def cosine_similarity(x, y):
  '''
  Calculates and returns a single cosine similarity value for x and y
  '''
  return 1 - spatial.distance.cosine(x, y)

def calculate_cosine_similarity(reference, holdout):
  '''
  Returns a numpy array of cosine similarity values for reference and holdout
  '''
  cosine_similarities = []
  for i in range(holdout.shape[0]):
    holdout_value = holdout[i,:]
    reference_value = reference[i,:]
    cosine_similarity_value = cosine_similarity(holdout_value, reference_value)
    cosine_similarities.append(cosine_similarity_value)
  return np.array(cosine_similarities)

##IV. Adjustments

### Mean Intensity

* Our dataset contains rows with the same `peptide_sequence`,  `precursor_charge`, `ion type` and `no` values, but different `intensity` values

<img src='pictures_and_graphs/redundancy.JPG'>

* **Idea**: calculate mean value for auch `peptide_sequence` + `precursor_charge` + `ion_type` + `no` combination => consequence: smaller dataset for training

THIS WAS DONE BEFORE STEP 4 OF PREPROCESSING ⤵

In [ ]:
def reduce_noise(df):
  '''
  Takes df with different intensity values for 'petide seq' + 'precursor charge' + 
  'ion_type' + 'no' combination, calculates mean intensity value for each 
  combination and returns the preprocessed df with new 
  'mean_normalized_intensity' column
  '''
  # Reducing the noise => one normalized intensity value for the same 'petide seq' + 'precursor charge' + 'ion_type' + 'no' combination
  result = df.groupby(['peptide_sequence', 'precursor_charge', 'ion_type', 'no'])['normalized_intensity'].mean().reset_index()
  result.rename(columns={'normalized_intensity': 'mean_normalized_intensity'})
  return result

* While using create_target() function few adjustments were made (commented out code):
  * `group`column: df was grouped by `peptide_sequence` and `precursor_charge`
  * `normalized_intensity` was replaced with newly created `mean_normalized_intensity`

**RESULTS**

<img src='pictures_and_graphs/mean_intensity_preprocessing_performance.png' width=400>

### Considering Vs Ignoring Methionine Modification
* Methionine was present in two forms with our dataset:
  * modified
  * unmodified

* **Idea**: take the same dataset, divide it into 5 cross-validation sets and a test set. In first case replace methionine and in second don't. Evaluate the unmodified cross-validation splits on unmodified test set & evaluate the modified cross-validation splits on modified test set. Compare performance

* Steps 1-3 are the same. Then first the splitting into training and test is done as described in `methionine_train_test_split` function

* There are 4 groups present in our dataset ragarding *methionine contents*:  1. Contains just unmodified methionine
  2. Contains just modified methionine
  3. Contains both
  4. Doesn't contain any methionine


* We took 20% of groups 1-3 as a test set. Since group 3 was the smallest in size, we left its test set as is, but we made sure that test sets we got from groups 1 and 2 were equal in size 


In [ ]:
def equal_sample(df1, df2):
  '''
  Compares two dataframes in size, takes a sample of the same len as the smaller 
  dataframe from the bigger one, returns two dataframes equal in size
  '''
  if len(df1) > len(df2):
    sample_size = len(df2)
    sampled_df = df1.sample(n=sample_size, replace=False)
  else:
    sample_size = len(df1)
    sampled_df = df2.sample(n=sample_size, replace=False)
  return df1, df2

def methionine_train_test_split(df, test_size=0.2):
  '''
  Splits df into three subsets: group1, group2 and group3. Takes test_size from 
  each df, makes sure that tests made from group1 and group2 dfs are of equal size,
  concatenates three test_sets. That is the new test df, removes 'peptide_sequence' 
  values from df. That is the training df. Returns test and training
  '''
  df_unmod = df.loc[(df['peptide_sequence'].str.contains('M')) & 
                    ~(df['peptide_sequence_modified'].str.contains('O'))]
  df_mod = df.loc[(df['peptide_sequence'].str.contains('O')) & 
                  ~(df['peptide_sequence_modified'].str.contains('M'))]
  df_both = df.loc[(df['peptide_sequence'].str.contains('O')) & 
                   (df['peptide_sequence_modified'].str.contains('M'))]

  training_unmod, test_unmod = split_train_test(df_unmod, test_size)
  training_mod, test_mod = split_train_test(df_mod, test_size)
  test_mod, test_unmod = equal_sample(test_mod, test_unmod)
  
  training_both, test_both = split_train_test(df_both, test_size)

  test = pd.concat([test_mod, test_unmod, test_both])
  training = df[~df['peptide_sequence'].isin(test['peptide_sequence'])]
  return training, test

* After this the same training and test sets were preprocessed in a different way:
  * Considering Modification Splits: encode_peptides, save_test, split_cross_val
  * Ignoring Modification Splits: encode_peptides (`aa_dict without '0'`, only 20 letters get replaced with numbers), save_test, split_cross_val 

**RESULTS**

<img src ='pictures_and_graphs/methionine.png' width=600>

### Proline

* has an impact on fragmentation (see heatmap in Step 2 Preprocessing)
* **Idea**: train a model on the dataset, which contains both sequences with and without proline and evaluate it on two different holdout sets:
    * holdout set with every sequence containing at least one proline
    * holdout set with every sequence not containing any proline
* all steps 1-4 stay the same, changes in step 5 ⤵

In [ ]:
def create_proline_splits(df):
  '''
  Takes dataframe, sorts out the rows which contain at least one proline (df_proline)
  and rows without proline at all (df_no_proline), takes 10% as test set from 
  each of new subsets, makes sure that training and test sets from each subset 
  are equal in size, concatenates training sets.
  Returns test_proline, test_no_proline and training dataframes
  '''
  df_proline = df.loc[df['peptide_sequence'].str.contains("P")]
  df_no_proline = df.loc[~df['peptide_sequence'].str.contains("P")]

  training_proline, test_proline = split_train_test(df_proline, 0.1)
  training_no_proline, test_no_proline = split_train_test(df_no_proline, 0.1)

  test_proline, test_no_proline = equal_sample(test_proline, test_no_proline)

  training_proline, training_no_proline = equal_sample(training_proline, training_no_proline)
  training = pd.concat([training_proline, training_no_proline])
  return test_proline, test_no_proline, training

**RESULTS**

<img src='pictures_and_graphs/proline.png' width=600>

Two-Sample T-Test proves that results are significantly different and didn't occur by chance

<img src='pictures_and_graphs/two_sample_t_test.png' width=450>

In [19]:
import scipy.stats as stats
def two_sample_t_test(predictions1, predictions2):
  '''
  Takes numpy arrays predictions1 and predictions2, runs two-sample t-test and
  prints whether the results are significantly different based on 0.05 threshold
  '''
  # Conduct the two-sample t-test to compare the results from the two predictive analyses
  t_statistic, p_value = stats.ttest_ind(predictions1, predictions2)

  # Determine the significance of the difference between the results obtained from the two predictive analyses
  if np.all(p_value < 0.05):
      print("The results obtained are significantly different (p-value = {})".format(p_value))
  else:
      print("The results obtained are not significantly different (p-value = {})".format(p_value))